# 🎵 LSTM vs Transformer: 감정 → 가사 → 음악 End-to-End (KOR+ENG)

이 노트북은 다음 파이프라인을 한 번에 실행합니다:
1) **데이터 다운로드 & 전처리** (IMDB + NSMC / Genius Lyrics / JSB Chorales)
2) **감정 분석**: LSTM vs Transformer 분류기 비교
3) **가사 생성**: LSTM LM vs GPT-like Transformer LM 비교 (한/영 혼합)
4) **음악 생성(멜로디)**: LSTM vs Transformer 비교 + WAV 생성

👉 *Colab 런타임(GPU) 권장*

## 0) 환경 준비

In [ ]:

# (권장) GPU 확인
import torch, platform, os, sys
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())

# 필수 패키지 설치
!pip -q install sentencepiece tqdm pandas numpy scikit-learn scipy

# (선택) Kaggle CLI 설치 (Colab에선 사전 설치됨)
!pip -q install kaggle

# Kaggle API 키 설정되어 있어야 합니다.
# (Colab) 아래 주석 제거하고 실행 후, kaggle.json 업로드
# from google.colab import files
# files.upload()  # kaggle.json 업로드
# !mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


## 1) 프로젝트 스크립트 업로드(또는 복사)
이 노트북은 `data_prep.py`, `train.py`를 현재 런타임으로 복사해서 사용합니다.

In [ ]:

import os, shutil, pathlib

workspace = "/content/ai_song_project"
os.makedirs(workspace, exist_ok=True)

# 만약 파일을 업로드했으면, 이 셀에서 경로 수정해서 복사해 쓰세요.
# 여기서는 노트북과 같은 폴더에 있다고 가정해 복사합니다.
srcs = ["data_prep.py", "train.py"]
for s in srcs:
    if os.path.exists(s):
        shutil.copy(s, workspace)
    else:
        print("경고: 현재 디렉토리에", s, "가 없습니다. 직접 업로드하거나, 파일 경로를 수정하세요.")

%cd $workspace
!ls -al


## 2) 데이터 다운로드 & 전처리

In [ ]:

# 데이터/토크나이저 준비
!python data_prep.py --data_dir ./data --download --spm_vocab_size 8000 --max_lyrics_per_lang 5000


## 3) 학습: 감정 분석 (LSTM vs Transformer)

In [ ]:

# LSTM 분류기
!python train.py train_sentiment --model_type lstm --data_dir ./data --out_dir ./outputs --epochs 3 --batch_size 64 --d_model 256

# Transformer 분류기
!python train.py train_sentiment --model_type transformer --data_dir ./data --out_dir ./outputs --epochs 3 --batch_size 64 --d_model 256 --n_layers 4 --n_heads 4 --dim_ff 512


## 4) 학습: 가사 생성 (LSTM LM vs GPT-like Transformer LM)

In [ ]:

# LSTM LM
!python train.py train_lyrics --model_type lstm --data_dir ./data --out_dir ./outputs --epochs 3 --batch_size 32 --d_model 384 --n_layers 2

# Transformer LM
!python train.py train_lyrics --model_type transformer --data_dir ./data --out_dir ./outputs --epochs 3 --batch_size 32 --d_model 384 --n_layers 4 --n_heads 6 --dim_ff 1024


## 5) 학습: 음악(멜로디) 생성 (LSTM vs Transformer)

In [ ]:

# Note-LSTM
!python train.py train_music --model_type lstm --data_dir ./data --out_dir ./outputs --epochs 3 --batch_size 64 --d_model 256 --n_layers 2

# Note-Transformer
!python train.py train_music --model_type transformer --data_dir ./data --out_dir ./outputs --epochs 3 --batch_size 64 --d_model 256 --n_layers 4 --n_heads 4 --dim_ff 512


## 6) End-to-End Demo: 상황 → 감정 → 가사 → 음악

In [ ]:

from IPython.display import Audio, display

input_text = "오늘은 오랜만에 친구들과 밤바다를 보러 가서 너무 신난다!"

!python train.py infer \
  --input_text "$input_text" \
  --sentiment_lstm ./outputs/sentiment_lstm.pth \
  --sentiment_tr   ./outputs/sentiment_transformer.pth \
  --lyrics_lstm    ./outputs/lyrics_lstm.pth \
  --lyrics_tr      ./outputs/lyrics_transformer.pth \
  --music_lstm     ./outputs/music_lstm.pth \
  --music_tr       ./outputs/music_transformer.pth \
  --out_dir        ./outputs

# 오디오 재생 (노트북에서)
display(Audio(filename="./outputs/song_lstm.wav"))
display(Audio(filename="./outputs/song_tr.wav"))
